In [2]:
import tensorflow as tf
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from argparse import ArgumentParser
from configparser import ConfigParser
from keras.models import Sequential
from keras.layers import Activation, Dense, AlphaDropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.utils.generic_utils import get_custom_objects
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import compat
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress Keras/TF warnings
compat.v1.logging.set_verbosity(compat.v1.logging.ERROR) # suppress Keras/TF warnings

In [3]:
tree = Signal('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root')

/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root


In [4]:
id_sort = ak.argsort(tree.jet_signalId, axis=1, ascending=False)[:,:6]
tree.jet_signalId[id_sort]

<Array [[5, 4, 3, 2, 1, 0, ... 3, 2, 1, 0, -1]] type='569341 * var * int32'>

In [5]:
btag_sort = ak.argsort(tree.jet_btag, axis=1, ascending=False)[:,:6]
id_sorted_by_btag = tree.jet_signalId[btag_sort]

In [6]:
jets = []

for i in range(6):
    jet = Particle(kin_dict={
            'pt' : tree.jet_ptRegressed[btag_sort][:,i].to_numpy(),
            'eta' : tree.jet_eta[btag_sort][:,i].to_numpy(),
            'phi' : tree.jet_phi[btag_sort][:,i].to_numpy(),
            'm' : tree.jet_mRegressed[btag_sort][:,i].to_numpy()
            })
    jets.append(jet)

deltaR = []
for i in range(5):
    for j in range(i+1,6):
        deltaR.append(jets[i].deltaR(jets[j]))

deltaR = np.column_stack((deltaR))


In [7]:
np.column_stack(([jet.pt for jet in jets])).shape, np.column_stack(([jet.eta for jet in jets])).shape, deltaR.shape

((569341, 6), (569341, 6), (569341, 15))

In [8]:
features = np.column_stack((
    np.column_stack(([jet.pt for jet in jets])),
    np.column_stack(([jet.eta for jet in jets])),
    deltaR
))

In [9]:
n_inputs = features.shape[1]
features.shape

(569341, 27)

In [38]:
indices = []
labels = []

for i in range(5):
    for j in range(i+1,6):
        jet1_HX_b1 = id_sorted_by_btag[:,i] == 0
        jet2_HX_b2 = id_sorted_by_btag[:,j] == 1
        jet12_HX_b1_b2 = jet1_HX_b1 & jet2_HX_b2

        jet1_HX_b2 = id_sorted_by_btag[:,i] == 1
        jet2_HX_b1 = id_sorted_by_btag[:,j] == 0
        jet12_HX_b2_b1 = jet1_HX_b2 & jet2_HX_b1

        mask = (jet12_HX_b1_b2 | jet12_HX_b2_b1).to_numpy()
        
        indices.append([i,j])
        labels.append(mask*1)

labels = np.column_stack((labels))

HX_not_found_mask = labels.sum(axis=1) == 0
HX_not_found = HX_not_found_mask*1
labels = np.column_stack((labels, HX_not_found))

assert np.all(labels.sum(axis=1) < 2)


n_outputs = labels.shape[1]
indices, labels, labels.shape

([[0, 1],
  [0, 2],
  [0, 3],
  [0, 4],
  [0, 5],
  [1, 2],
  [1, 3],
  [1, 4],
  [1, 5],
  [2, 3],
  [2, 4],
  [2, 5],
  [3, 4],
  [3, 5],
  [4, 5]],
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]),
 (569341, 16))

In [43]:
n_HX_not_found_goal = int(np.average(labels.sum(axis=0)[:-1]))

In [39]:
HX_not_found_mask

array([False, False, False, ..., False, False, False])

In [55]:
HX_not_found_ind = np.arange(len(features))[HX_not_found_mask][:n_HX_not_found_goal]
HX_found_ind = np.arange(len(features))[~HX_not_found_mask]

feature_indices = np.concatenate((HX_not_found_ind, HX_found_ind))

features = features[feature_indices]
labels = labels[feature_indices]

In [74]:
labels.sum(axis=0)/labels.sum(axis=0).sum()

array([0.08853328, 0.08759225, 0.08279078, 0.06669202, 0.03670507,
       0.08633261, 0.08162252, 0.06590166, 0.03591964, 0.08106433,
       0.06469635, 0.0353244 , 0.06293532, 0.03351396, 0.02787767,
       0.06249815])

In [67]:
# Hidden hyperparameters
hidden_activations = 'selu'
nodes              = [20,20]
nlayers            = len(nodes)

# Output hyperparameters
output_activation  = 'softmax'
output_nodes       = n_outputs

optimizer_name     = 'nadam'
loss_function      = 'categorical_crossentropy'
nepochs            = 50
batch_size         = 50

In [68]:
scaler = MinMaxScaler()
scaler.fit(features)
x = scaler.transform(features)

val_size = 0.10
x_train, x_val, y_train, y_val = train_test_split(x, labels, test_size=val_size)

param_dim = n_inputs

In [69]:
model = Sequential()

model.add(Dense(nodes[0], input_dim=param_dim, activation=hidden_activations))
for i in range(1,nlayers):
    model.add(Dense(int(nodes[i]), activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    # model.add(Dense(int(nodes[i]), kernel_initializer='lecun_normal', activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    # if bool(args.dropout): model.add(AlphaDropout(0.2)) 
model.add(Dense(output_nodes, activation=output_activation))

In [70]:
lr = 0.001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-7

es = EarlyStopping(monitor='loss', restore_best_weights=True, patience=5)
met = ['accuracy']

optimizer = tf.keras.optimizers.Nadam(
    learning_rate=lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, name="nadam"
)

model.compile(loss=loss_function, 
              optimizer=optimizer, 
              metrics=met)

In [75]:
history = model.fit(x_train,
                    y_train, 
                    validation_data=(x_val, y_val), 
                    epochs=nepochs, 
                    batch_size=batch_size, 
                    callbacks=[es])

model.summary()

Epoch 1/50
1822/1822 [==============================] - 9s 5ms/step - loss: 2.4324 - accuracy: 0.1954 - val_loss: 2.4130 - val_accuracy: 0.1935
Epoch 2/50
1822/1822 [==============================] - 9s 5ms/step - loss: 2.4011 - accuracy: 0.1964 - val_loss: 2.3889 - val_accuracy: 0.1953
Epoch 3/50
1822/1822 [==============================] - 9s 5ms/step - loss: 2.3787 - accuracy: 0.1956 - val_loss: 2.3724 - val_accuracy: 0.1928
Epoch 4/50
 536/1822 [=======>......................] - ETA: 6s - loss: 2.3686 - accuracy: 0.1967

KeyboardInterrupt: 